# Ch6. Beyond Born-Oppenheimer

## Install

In [ ]:
#Installment

## 6.1. Non-Born-Oppenheimer molecular Hamiltonian

### Internal Hamiltonian operator

#### Explicitly correlated all-particle Gaussian functions

### Energy minimization

## 6.2. Vibrational frequency analysis calculations

### Modeling the vibrational-rotational levels of a diatomic molecule

In [ ]:
# Modeling a diatomic molecule

##### Figure 6.5 - Potential energy(black parabola) and spatial probabilites (color) of the normalized quantum harmonic oscillator wavefunctions for $ v = 0 $ to $ v = 4 $

In [ ]:
# Modeling a diatomic molecule

##### Figure 6.6 - Vibrational energy levels of the hydrogen molecule for $v = 0, \ldots, 0, J = 0$ and $J = 1$

### Computing all vibrational-rotational levels of a molecule

#### Optimizing the geometry of the molecule

#### Calculating a force constant Hessian matrix

#### Converting to mass weighted Cartesian coordinates

#### Diagnalizing the mass weightened Hesssian matrix

In [ ]:
# Code here?

## 6.3. Vibrational spectra for ortho-para isomerization of hydrogen molecules

In [ ]:
# Code here?

## Summary